In [10]:
import requests
import json
from pprint import pprint
import pandas as pd
import networkx as nx

In [2]:
player_dict = {}
nba_teams = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 17, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 38, 40, 41]
g = nx.Graph()
for x in range(0,3):
    for y in nba_teams:
        
        url = f"https://api-nba-v1.p.rapidapi.com/players?season=202{x}&team={y}" 

        headers = {
            "X-RapidAPI-Key": "10755b2a08msh15dff173eafe850p158f43jsnf3d99db20514",
            "X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers)

        response_dict = response.json()


        for z in range(len(response_dict['response'])):
            current_id = response_dict['response'][z]['id']
            if current_id not in player_dict:
                player_dict[current_id] = {'name':[response_dict['response'][z]['firstname'], response_dict['response'][z]['lastname']],
                                           'teams': [y], 
                                           'active':False}
                if x == 2:
                    player_dict[current_id]['active'] = True 
                    g.add_node(current_id)
            else:
                player_dict[current_id]['teams'].append(y)
                if x == 2:
                    player_dict[current_id]['active'] = True 

In [3]:
json_object = json.dumps(player_dict, indent=3)
with open('players.json', 'w') as file:
    file.write(json_object)

# Running Without API

In [3]:
#RUNNING WITHOUT API
player_dict = {}
nba_teams = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 17, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 38, 40, 41]
g = nx.Graph()
with open('players.json', 'r') as file:
    player_dict = json.load(file)
for key in player_dict:
    if(player_dict[key]['active']):
        g.add_node(key)

In [4]:
count = 0
for player in player_dict:
    if(player_dict[player]['active']):
        count +=1
print(count, len(player_dict))

601 1047


In [13]:
team_dict = {}
for team in nba_teams:
    
    url1 = f"https://api-nba-v1.p.rapidapi.com/teams?id={team}" 

    headers = {
        "X-RapidAPI-Key": "10755b2a08msh15dff173eafe850p158f43jsnf3d99db20514",
        "X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
    }

    team_response = requests.get(url1, headers=headers)
    

    team_response_dict = team_response.json()
    
    
    team_dict[team] = team_response_dict['response'][0]['name']

In [6]:
# team_dict = {}
# json_object = json.dumps(team_dict)
# with open('teams.json', 'w') as file:
#     file.write(json_object)

In [14]:
with open('teams.json', 'r') as file:
    team_dict = json.load(file)
#print(team_dict)

In [15]:
player_ids = list(player_dict.keys())
for left_player in range(len(player_ids)):
    if(player_dict[player_ids[left_player]]['active']):
        for right_player in range(left_player + 1, len(player_ids)):
            if(player_dict[player_ids[right_player]]['active'] and not set(player_dict[player_ids[left_player]]['teams']).isdisjoint(set(player_dict[player_ids[right_player]]['teams']))):
                    current_weight = g.get_edge_data(left_player, right_player, default={"weight":0})["weight"]
                    g.add_edge(player_ids[left_player], player_ids[right_player])


In [41]:
re_edge = list()
player = '3195'
edges = list(g.edges(player))
print(f"{player_dict[player]['name'][0]} {player_dict[player]['name'][1]} has played with {len(edges)} active players")
for edge in edges:
    re_edge.append((player_dict[edge[0]]['name'], player_dict[edge[1]]['name']))
pprint(re_edge)

Graph with 601 nodes and 19363 edges
Matt Ryan has played with 165 active players
[(['Matt', 'Ryan'], ['Bruno', 'Fernando']),
 (['Matt', 'Ryan'], ['Nathan', 'Knight']),
 (['Matt', 'Ryan'], ['Danilo', 'Gallinari']),
 (['Matt', 'Ryan'], ['Jaylen', 'Brown']),
 (['Matt', 'Ryan'], ['Jayson', 'Tatum']),
 (['Matt', 'Ryan'], ['Luke', 'Kornet']),
 (['Matt', 'Ryan'], ['Daniel', 'Theis']),
 (['Matt', 'Ryan'], ['Moritz', 'Wagner']),
 (['Matt', 'Ryan'], ['Robert', 'Williams III']),
 (['Matt', 'Ryan'], ['Romeo', 'Langford']),
 (['Matt', 'Ryan'], ['Grant', 'Williams']),
 (['Matt', 'Ryan'], ['Javonte', 'Green']),
 (['Matt', 'Ryan'], ['Aaron', 'Nesmith']),
 (['Matt', 'Ryan'], ['Payton', 'Pritchard']),
 (['Matt', 'Ryan'], ['Kemba', 'Walker']),
 (['Matt', 'Ryan'], ['Evan', 'Fournier']),
 (['Matt', 'Ryan'], ['Marcus', 'Smart']),
 (['Matt', 'Ryan'], ['Caris', 'LeVert']),
 (['Matt', 'Ryan'], ['Taurean', 'Prince']),
 (['Matt', 'Ryan'], ['Jarrett', 'Allen']),
 (['Matt', 'Ryan'], ['Bruce', 'Brown']),
 (['Matt'

In [17]:
betweeness = nx.betweenness_centrality(g)
nx.set_node_attributes(g, betweeness, 'betweenness')

In [32]:
nx.write_graphml(g, 'players.graphml')

In [27]:
df_dict = dict()
for pid in betweeness:
    df_dict[pid] = {'Name': player_dict[pid]['name'][0] + ' ' + player_dict[pid]['name'][1],
                    'Centrality': betweeness[pid]}
df = pd.DataFrame(df_dict.values(), index=df_dict.keys(), columns=['Name', 'Centrality'])
df.sort_values(by='Centrality', ascending=False, inplace=True)
df.head(25)

,Name,Centrality
626,Shaquille Harrison,0.022477
2391,Chris Silva,0.018791
983,Alize Johnson,0.017289
1006,Svi Mykhailiuk,0.015102
802,Wes Iwundu,0.014409
2609,Anthony Lamb,0.011728
2623,Sam Merrill,0.011204
203,Danny Green,0.010995
970,Wenyen Gabriel,0.010834
2077,Zylan Cheatham,0.010432


In [21]:
print(f"1:{player_dict['626']}\n2:{player_dict['2391']}\n3{player_dict['983']}")

1:{'name': ['Shaquille', 'Harrison'], 'teams': [9, 40, 4, 19, 27, 17], 'active': True}
2:{'name': ['Chris', 'Silva'], 'teams': [20, 30, 4, 20, 22, 1, 8], 'active': True}
3{'name': ['Alize', 'Johnson'], 'teams': [4, 38, 4, 6, 23, 41, 31], 'active': True}


In [22]:
ego = nx.ego_graph(g, '626', radius=1)

In [19]:
df.to_csv('player_centrality.csv')

In [178]:
nx.write_graphml(ego, "shaq_har.graphml")